In [0]:
import random
import sklearn
from operator import add
import math
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from sklearn.preprocessing import MinMaxScaler

In [0]:
iris = "/FileStore/tables/iris.csv"
glass = "/FileStore/tables/glass.csv"
parkinsons = "/FileStore/tables/parkinsons.csv"
file_type = "csv"
iris = spark.read.csv(iris,header=True,inferSchema=True)
park = spark.read.csv(parkinsons,header=True,inferSchema=True)
glass = spark.read.csv(glass,header=True,inferSchema=True)


In [0]:
def preprocess(df):
    df=df.toPandas()
    scale_cols=df.columns[:-1] #choosing columns to perform min max on
    df_class=[]
    class_col=df.columns[-1:] #choosing class column
    t=np.array(df[class_col]) #saving class values as a list
    for i in range(len(t)):
        df_class.append(t[i][0])
    scaler=MinMaxScaler() #performing min max standartization
    scaler.fit(df[scale_cols])
    df = pd.DataFrame(scaler.transform(df[scale_cols]),columns=df[scale_cols].columns)
    df = spark.createDataFrame(df)
    return df,df_class #returning standartized df and classes list
    

In [0]:
# a func that the map will apply on the data that receives the dataframe and the a list of random centroids
def f_for_map(p1,centroids):
    dmin=math.inf #searching for min dist, therefore initializing to inf
    idx = -1 #initializing of minimal index to -1
    for j in range(len(centroids)): #iterating through the centroids list
        d = np.linalg.norm(p1-centroids[j]) #calculating distance of the current row to the current centroid
        if(d<dmin): #saving minimal dist index and value
            dmin=d
            idx = j
    return(idx,p1)#returns a tapple with index of centroid and the assigned row

In [0]:
def avg_reduce(k,c_dict): 
    return (k[0],k[1]/c_dict[k[0]]) # a reduce function to calculate the average of the 

In [0]:
#the function that performs k-means algorithm as requested with default vals
def kmeans(df,k,ct=0.0001,i=30,exp=10):
    res=preprocess(df)#preprocessing function
    class_list=res[1] #output of processing
    ch_list=[]
    ari_list=[]
    for w in range(exp): #experiment iteration
        dist_sum=math.inf #searching to be less then threshhold - ct , so initializing to inf
        for j in range(i): #within experiment iterations
            if j==0: # at first iteration the k initial centroids are random
                centroids = res[0].rdd.takeSample(False,k)
            kv_list=res[0].rdd.map(lambda row:f_for_map(np.array(row),centroids)) #applying map on the dataframe with f_for_map func that assigns rows to centroids
            count_dict=kv_list.countByKey() #calculating number of df rows assigned to each centroid
            kv_list=kv_list.map(lambda row:(row[0],np.array(row[1]))) #processing data to be in a shape that fits a reduce func
            preds=kv_list.map(lambda row:row[0]) #extracting predicted values per row
            red_kv_list=kv_list.reduceByKey(lambda a,b:a+b) #reduce func that summs all the rows per key by thier dimensions
            new_centroids=red_kv_list.map(lambda k:avg_reduce(k,count_dict)).collect() #a mapper that dives summs with the count value in order to calc average per key
            coords=[]
            for z in range(len(new_centroids)): #extracting dimensions of the new calculated centroids
                coords.append(new_centroids[z][1])
            s=0 #initialization of sum variable to sum the distances from old centroids to new ones
            q=[] #initialization 
            for d in range(len(centroids)):
                for f in range(len(centroids[d])):
                    s+=abs(centroids[d][f]-coords[d][f])
            dis_sum=s #updating condition sum
            if(dis_sum<ct): #checking convergens condition
                break
        centroids=coords #updating centroids variable to be the new calculated centroids
        ch=sklearn.metrics.calinski_harabasz_score(df.rdd.collect(),preds.collect()) #calculation of CH score for each experiment
        ari=sklearn.metrics.adjusted_rand_score(class_list,preds.collect()) #calculation of ARI score for each experiment
        ch_list.append(ch) #saving results to lists in order to calculate mean and std later
        ari_list.append(ari) 
    print("The Mean and Standard Deviation of Calinski-Harabasz score for " + str(k) + " clusters are: " + str((np.array(ch_list).mean(),np.array(ch_list).std())) + "\n" + "The Mean and Standard Deviation of Adjusted-Rand-Index score for " + str(k) + " clusters are: " + str((np.array(ari_list).mean(),(np.array(ari_list).std()) )))
  
        

In [0]:
kmeans(park,5)

The Mean and Standard Deviation of Calinski-Harabasz score for 5 clusters are: (24.432037994644947, 13.478368369354003)
The Mean and Standard Deviation of Adjusted-Rand-Index score for 5 clusters are: (0.1030855689294409, 0.04082675950860078)
